<a href="https://colab.research.google.com/github/llawlaw23/Recommendation-System/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import os
import gc
import re

In [44]:
props1 = pd.read_csv("Datasets/item_properties_part1.1.csv")
props2 = pd.read_csv("Datasets/item_properties_part2.csv")
props = pd.concat([props1, props2])

In [45]:
df_items = props[props["property"] == "categoryid"][["itemid", "value"]]

In [49]:
df_items = df_items.rename(columns={"value": "categoryid"})

In [50]:
df_items.head()

,itemid,categoryid
0,460429,1338
140,281245,1277
151,35575,1059
189,8313,1147
197,55102,47


In [51]:
df_items["categoryid"] = df_items["categoryid"].astype("Int64")

In [52]:
df_items.shape

(788214, 2)

In [53]:
# saved the clean csv as an new file in the cleaned folder and delete the previous one
df_items.to_csv("df_item.csv", index = False)
del df_items; gc.collect()

269

In [7]:
props.shape

(20275902, 4)

In [ ]:
props.head()

,timestamp,itemid,property,value
0,1435460400000,460429,categoryid,1338
1,1441508400000,206783,888,1116713 960601 n277.200
2,1439089200000,395014,400,n552.000 639502 n720.000 424566
3,1431226800000,59481,790,n15360.000
4,1431831600000,156781,917,828513


In [41]:
events = pd.read_csv("Datasets/events.csv")
cats = pd.read_csv("category_tree.csv")

In [42]:
cats.head()

,categoryid,parentid
0,1016,213.0
1,809,169.0
2,570,9.0
3,1691,885.0
4,536,1691.0


### Data Understanding & Preprocessing

#### 1. Data Overview
- Brief description of the dataset ( size, number of features, number of columns and rows, and observations)
- Types of variables (numerical, categorical, datetime, etc.)
- Initial observations about the data (e.g., imbalance, missing data, outliers)

#### 2. Data Quality Checks
- Check for missing values and filling or dropping them.
- dropping columns.
- checking for outliers.

In [ ]:
events["timestamp"] = pd.to_datetime(events["timestamp"], unit = "ms")

In [10]:
events.shape

(2756101, 5)

In [ ]:
events.isnull().sum()

,0
timestamp,0
visitorid,0
event,0
itemid,0
transactionid,2733644


In [11]:
events = events.drop(columns = ["transactionid"])

In [ ]:
events.head()

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,view,355908
1,2015-06-02 05:50:14.164,992329,view,248676
2,2015-06-02 05:13:19.827,111016,view,318965
3,2015-06-02 05:12:35.914,483717,view,253185
4,2015-06-02 05:02:17.106,951259,view,367447


In [12]:
# map view = 1 add to chart = 3 and purchase  = 5, replacing it in the old column. dataype to int
event_weights = {"view": 1, "addtocart": 3, "transaction": 5}
events["event"] = events["event"].map(event_weights)

In [13]:
# change datatype of event to int
events["event"] = events["event"].fillna(0).astype(int)

In [ ]:
events.head(20)

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,1,355908
1,2015-06-02 05:50:14.164,992329,1,248676
2,2015-06-02 05:13:19.827,111016,1,318965
3,2015-06-02 05:12:35.914,483717,1,253185
4,2015-06-02 05:02:17.106,951259,1,367447
5,2015-06-02 05:48:06.234,972639,1,22556
6,2015-06-02 05:12:03.240,810725,1,443030
7,2015-06-02 05:34:51.897,794181,1,439202
8,2015-06-02 04:54:59.221,824915,1,428805
9,2015-06-02 05:00:04.592,339335,1,82389


In [ ]:
# min in visitorid
events["visitorid"].min()

0

checking for outliers and clearing them

In [14]:
visitor_event = events.groupby("visitorid")["event"].count()

In [ ]:
# print in descending order
visitor_event = visitor_event.sort_values(ascending = False)

In [ ]:
visitor_event.head(20)

,event
visitorid,
1150086,7757
530559,4328
152963,3024
895999,2474
163561,2410
371606,2345
286616,2252
684514,2246
892013,2024


In [15]:
Q1 = visitor_event.quantile(0.25)
Q3 = visitor_event.quantile(0.75)
IQR = Q3 - Q1

In [16]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [17]:
upper_bound

3.5

In [112]:
lower_bound

np.float64(-0.5)

In [ ]:
outlier_visitor_ids = visitor_event[(visitor_event < lower_bound) | (visitor_event > upper_bound)].index

In [ ]:
is_outlier = events["visitorid"].isin(outlier_visitor_ids)

In [20]:
print (is_outlier)

0          False
1           True
2          False
3          False
4          False
           ...  
2756096    False
2756097    False
2756098    False
2756099    False
2756100    False
Name: visitorid, Length: 2756101, dtype: bool


In [21]:
events_F = events[~is_outlier]

In [117]:
events_F.head()

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,1,355908
2,2015-06-02 05:13:19.827,111016,1,318965
3,2015-06-02 05:12:35.914,483717,1,253185
4,2015-06-02 05:02:17.106,951259,1,367447
5,2015-06-02 05:48:06.234,972639,1,22556


In [22]:
events_F.shape

(1652380, 4)

In [23]:
props.head()

,timestamp,itemid,property,value
0,2015-06-28 03:00:00,460429,categoryid,1338
1,2015-09-06 03:00:00,206783,888,1116713 960601 n277.200
2,2015-08-09 03:00:00,395014,400,n552.000 639502 n720.000 424566
3,2015-05-10 03:00:00,59481,790,n15360.000
4,2015-05-17 03:00:00,156781,917,828513


In [22]:
itemid_value = props.groupby("itemid")["value"].count()

In [23]:
Q1 = itemid_value.quantile(0.25)
Q3 = itemid_value.quantile(0.75)
IQR = Q3 - Q1

In [24]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [25]:
outlier_itemid_value = itemid_value[(itemid_value < lower_bound) | (itemid_value > upper_bound)].index

In [26]:
is_outlier = props["itemid"].isin(outlier_itemid_value)

In [27]:
props = props[~is_outlier]

In [28]:
props.shape

(17520979, 4)

In [29]:
# clean the value column and keep values with n attached to them and add them if they are multiple
def sum_n_values(text):
    nums = re.findall(r"n([\d\.]+)", str(text))
    nums = [float(x) for x in nums]
    return sum(nums) if nums else None

props["value_n"] = props["value"].apply(sum_n_values)

In [30]:
props = props.drop(columns = ["value"])

In [34]:
props["timestamp"] = pd.to_datetime(props["timestamp"], unit = "ms")

In [35]:
props.head()

,timestamp,itemid,property,value_n
1,2015-09-06 03:00:00,206783,888,277.2
2,2015-08-09 03:00:00,395014,400,1272.0
3,2015-05-10 03:00:00,59481,790,15360.0
9,2015-06-14 03:00:00,98113,451,48.0
10,2015-08-09 03:00:00,450113,888,504.0


In [36]:
props.isnull().sum()

timestamp    0
itemid       0
property     0
value_n      0
dtype: int64

In [37]:
# drop null values in value_n column
props = props.dropna(subset = ["value_n"])

In [38]:
props.shape

(4300636, 4)

In [39]:
props.head()

,timestamp,itemid,property,value_n
1,2015-09-06 03:00:00,206783,888,277.2
2,2015-08-09 03:00:00,395014,400,1272.0
3,2015-05-10 03:00:00,59481,790,15360.0
9,2015-06-14 03:00:00,98113,451,48.0
10,2015-08-09 03:00:00,450113,888,504.0


In [62]:
cats.isnull().sum()

categoryid     0
parentid      25
dtype: int64

In [63]:
cats = cats.dropna()

In [40]:
# saved the clean csv as an new file in the cleaned folder and delete the previous one
props.to_csv("props_cleaned.csv", index = False)
del props; gc.collect()

60

In [64]:
# saved the clean csv as an new file in the cleaned folder and delete the previous one
events_F.to_csv("events_cleaned.csv", index = False)
del events_F; gc.collect()

1001

In [65]:
# saved the clean csv as an new file in the cleaned folder and delete the previous one
cats.to_csv("category_tree_cleaned.csv", index = False)
del cats; gc.collect()

0

In [16]:
# Loading cleaned csv files
props_cleaned = pd.read_csv("props_cleaned.csv")

events_cleaned = pd.read_csv("events_cleaned.csv")

cats_cleaned = pd.read_csv("cats_cleaned.csv")

Creating My user-item matric and item feature Matrix

In [68]:
from scipy.sparse import csr_matrix
import pandas as pd
# from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix
import numpy as np

In [70]:
events_cleaned.head()

,timestamp,visitorid,event,itemid
0,2015-06-02 05:02:12.117,257597,1,355908
1,2015-06-02 05:13:19.827,111016,1,318965
2,2015-06-02 05:12:35.914,483717,1,253185
3,2015-06-02 05:02:17.106,951259,1,367447
4,2015-06-02 05:48:06.234,972639,1,22556


In [ ]:
visitor2_idx = {v: i for i, v in enumerate(events_cleaned["visitorid"].unique())}
item2_idx = {v: i for i, v in enumerate(events_cleaned["itemid"].unique())}

In [ ]:
events_cleaned["visitor_idx"] = events_cleaned["visitorid"].map(visitor2_idx,)

events_cleaned["item_idx"] = events_cleaned["itemid"].map(item2_idx)

In [ ]:
user_item_matrix = coo_matrix(
    (events_cleaned["event"], (events_cleaned["visitor_idx"], events_cleaned["item_idx"]))
)

In [ ]:
events_computed_agg = events_F.groupby(["visitor_idx", "item_idx"])["interaction_strength"].sum().reset_index()

In [ ]:
events_computed = events_computed_agg.compute()

In [ ]:
rows = events_computed["visitor_idx"].values
cols = events_computed["item_idx"].values
data = events_computed["interaction_strength"].values

In [ ]:
user_item_matrix = csr_matrix((data, (rows, cols)), shape=(len(visitor_map), len(item_map)))

In [ ]:
print(f"Shape of the user-item interaction matrix: {user_item_matrix.shape}")

Shape of the user-item interaction matrix: (1287164, 210443)


In [ ]:
# categories = props["property"].unique().compute().tolist()

In [ ]:
# props["property"] = props["property"].astype(pd.CategoricalDtype(categories = categories))

In [ ]:
# item_feature_matrix = props.pivot_table(
#     index = "itemid",
#     columns = "property",
#     values = "value_n",
#     aggfunc = "first"
# )

Merging With Item properties and categories

In [ ]:
coo = user_item_matrix.tocoo()

In [ ]:
visitor_ids = events_F["visitorid"].unique().compute()
item_ids = events_F["itemid"].unique().compute()

In [ ]:
visitor2idx = {v: i for i, v in enumerate(visitor_ids)}
item2idx = {i: j for j, i in enumerate(item_ids)}

In [ ]:
events_idx = events_F.compute()
events_idx["visitor_idx"] = events_idx["visitor_idx"].astype("int32")
events_idx["item_idx"] = events_idx["item_idx"].astype("int32")

In [ ]:
events_idx = events_idx.dropna(subset=["visitor_idx", "item_idx"])

In [ ]:
coo = coo_matrix(
    (events_idx["interaction_strength"],
     (events_idx["visitor_idx"], events_idx["item_idx"]))
)

In [ ]:
df = pd.DataFrame({
    "visitor_idx": coo.row,
    "item_idx": coo.col,
    "interaction_strength": coo.data
})

In [ ]:
events_idx["visitor_idx"] = events_idx["visitorid"].map(visitor2idx)
events_idx["item_idx"] = events_idx["itemid"].map(item2idx)

In [ ]:
item_feature_matrix = item_feature_matrix.compute()

user_item_features = df.merge(
    item_feature_matrix,
    on="itemid",
    how="left"
)

MemoryError: Unable to allocate 1.87 GiB for an array with shape (251065907,) and data type int64

model fitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, test_df = train_test_split(
    user_item_features,
    test_size=0.2,
    random_state=42
)

In [ ]:

X_train = train_df.drop(["visitorid", "itemid", "interaction_strength"], axis=1)
y_train = train_df["interaction_strength"]

In [ ]:
X_test = test_df.drop(["visitorid", "itemid", "interaction_strength"], axis=1)
y_test = test_df["interaction_strength"]

In [ ]:
als_model = AlternatingLeastSquares(factors = 50, regularization = 0.1, iterations = 20)
als_model.fit(user_item_matrix)

In [ ]:
user_recs = {}
N = 10

for user_idx in range(user_item_matrix.shape[0]):
    recommended = als_model.recommend(user_idx, user_item_matrix[user_idx], N = N)
    user_recs[user_idx] = [(item_idx, score) for item_idx, score in recommended]

In [ ]:
recs_df = pd.DataFrame(
    [(user, item, score) for user, recs in user_recs.items() for item, score in recs],
    columns = ["visitor_idx", "item_idx", "score"]
)

#### Task 1 (Predict Add-to-Cart Properties)

In [ ]:
carts = events[events["event"] == "addtocart"]

### Task 2 (Detect Abnormal Users)

In [ ]:
unique_items = events.groupby("visitorid")["itemid"].nunique()

In [ ]:
event_counts = events.groupby("visitorid")["event"].count()

In [ ]:
user_stats = unique_items.to_frame("unique_items").merge(
    event_counts.to_frame("total_events"),
    left_index = True,
    right_index = True
)

In [ ]:
user_stats = user_stats.compute()

print(user_stats.head())

### Data Visualization and Analysing Business Question




- How do user demographics influence their preferences and interactions?

- What products are frequently purchased or viewed together?

- How do popularity metrics evolve monthly or seasonally?

- Which items frequently co-occur in transactions?

- How do preferences differ across age, location, or other demographics?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

Q1
- How many unique visitors per day and week?

In [ ]:
unique_visitors_daily = events.groupby(events["timestamp"].dt.date)["visitorid"].nunique().compute()

In [ ]:

print(unique_visitors_daily.head(10))

Q2
- What patterns exist in user-item interactions over time?